In [1]:
import os
import sys
import pandas as pd
import numpy as np

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
# Configuración de rutas para importar módulos locales
project_root = os.path.dirname(os.getcwd())
scripts_dir = os.path.join(project_root, "src", "scripts")
raw_data_inei = os.path.join(project_root, "data", "01_raw", "inei")
sys.path.insert(0, scripts_dir)

# 4. Local Application Imports
from getting_modules import EndesProcessor
from merging_data import JoinProcessor

from modeler_helper import (
    calcular_distancia,
    preparar_datos_bajo_peso,
    _basic_metrics,
    train_bajo_peso,

)



## Cargamos la data de Ubigeo para extrear variables socioeconómicas y geográficas 

### para luego calcular distancias del hogar hacia la capital de la provincia.

In [2]:
df_ubigeo = pd.read_csv(raw_data_inei + "\\geo\\ubigeo_provincia.csv", dtype={"inei": str})
df_ubigeo = df_ubigeo[
    [
        "inei",
        "departamento",
        "macroregion_inei",
        "pob_densidad_2020",
        "latitude",
        "longitude",
        "indice_densidad_estado",
        "indice_vulnerabilidad_alimentaria",
        "idh_2019",
        "pct_pobreza_total",
        "pct_pobreza_extrema",
    ]
]
df_ubigeo.rename(columns={"inei": "ubigeo_prov"}, inplace=True)

## Se extrea información importante de cada módulo de ENDES

In [ ]:
rutas_modulos_23 = [
    "910-Modulo1629",
    "910-Modulo1630",
    "910-Modulo1631",
    "910-Modulo1632",
    "910-Modulo1633",
    "910-Modulo1634",
    "910-Modulo1635",
    "910-Modulo1636",
    "910-Modulo1637",
    "910-Modulo1638",
    "910-Modulo1639",
    "910-Modulo1640",
    "910-Modulo1641",
]

rutas_modulos_24 = [
    "968-Modulo1629",
    "968-Modulo1630",
    "968-Modulo1631",
    "968-Modulo1632",
    "968-Modulo1633",
    "968-Modulo1634",
    "968-Modulo1635",
    "968-Modulo1636",
    "968-Modulo1637",
    "968-Modulo1638",
    "968-Modulo1639",
    "968-Modulo1640",
    "968-Modulo1641",
]


path_23 = raw_data_inei + "\\2023"
path_24 = raw_data_inei + "\\2024"


processor_23 = EndesProcessor(data_path=path_23, anio=2023)
processor_24 = EndesProcessor(data_path=path_24, anio=2024)


dfs_procesados_23 = {
    f"mod_{i + 1}": getattr(processor_23, f"procesar_modulo_{i + 1}")(
        os.path.join(path_23, rutas_modulos_23[i]), 2023
    )
    for i in range(len(rutas_modulos_23))
}

dfs_procesados_24 = {
    f"mod_{i + 1}": getattr(processor_24, f"procesar_modulo_{i + 1}")(
        os.path.join(path_24, rutas_modulos_24[i]), 2024
    )
    for i in range(len(rutas_modulos_23))
}

## Se concantena la data de 2 años para tener mayor información y se hace un proceso de limpieza con la función 'combinar_modulos'

In [4]:
join_processor_23 = JoinProcessor(dfs_procesados_23, "2023")
join_processor_24 = JoinProcessor(dfs_procesados_24, "2024")

df_master = pd.concat(
    [
        join_processor_24.combinar_modulos(dfs_procesados_24, "2024"),
        join_processor_23.combinar_modulos(dfs_procesados_23, "2023"),
    ],
    ignore_index=True,
)


print("shape antes de eliminar duplicados: ", df_master.shape)
id_cols = {
    "id_hogar",
    "id_miembro_hogar",
    "id_cuestionario_mujer",
    "id_nacimiento",
    "ubigeo",
    "anio",
    "latitud",
    "longitud",
}
# Nos aseguramos de que existan en el DF
id_cols = [c for c in id_cols if c in df_master.columns]

df_sin_ids = df_master.drop(columns=id_cols, errors="ignore")
df_master = df_master.loc[~df_sin_ids.duplicated(keep="first")].reset_index(drop=True)
df_master["ubigeo_prov"] = df_master["ubigeo"].astype(str).str[:-2] + "00"
df_master = df_master.merge(df_ubigeo, how="inner", on="ubigeo_prov")

df_master.drop(columns="ubigeo_prov", inplace=True)
df_master["distance_km_capital_province"] = df_master.apply(calcular_distancia, axis=1)

print("shape después  de eliminar duplicados: ", df_master.shape)

Uniendo con datos de características del hogar (Módulo 2)...
Traduciendo códigos a valores descriptivos...
Uniendo con datos prenatales y de parto (Módulo 5)...

Unión con Módulo 5 y mapeo completados.
shape antes de eliminar duplicados:  (19751, 92)
shape después  de eliminar duplicados:  (19751, 92)
Uniendo con datos de características del hogar (Módulo 2)...
Traduciendo códigos a valores descriptivos...
Uniendo con datos prenatales y de parto (Módulo 5)...

Unión con Módulo 5 y mapeo completados.
shape antes de eliminar duplicados:  (20840, 92)
shape después  de eliminar duplicados:  (20840, 92)
shape antes de eliminar duplicados:  (40591, 92)
shape después  de eliminar duplicados:  (40591, 103)


In [5]:
df_master.head()

,id_cuestionario_mujer,id_nacimiento,sexo_bebe,bebe_esta_vivo,intervalo_nacimiento_anterior_meses,id_miembro_hogar,total_hijos_nacidos,edad_mujer_primer_parto,mujer_actualmente_embarazada,mujer_uso_anticonceptivo_alguna_vez,...,macroregion_inei,pob_densidad_2020,latitude,longitude,indice_densidad_estado,indice_vulnerabilidad_alimentaria,idh_2019,pct_pobreza_total,pct_pobreza_extrema,distance_km_capital_province
0,325503101 2,1,2,1,60,3,3,22,0,3,...,ORIENTE,17.743489,-6.229444,-77.872778,0.767279,0.396175,0.542665,22.021474,11.698524,1.330599
1,325504701 2,1,2,1,130,4,2,20,0,3,...,ORIENTE,17.743489,-6.229444,-77.872778,0.767279,0.396175,0.542665,22.021474,11.698524,1.330599
2,325505001 1,1,2,1,216,3,2,17,0,3,...,ORIENTE,17.743489,-6.229444,-77.872778,0.767279,0.396175,0.542665,22.021474,11.698524,1.330599
3,325508901 2,1,2,1,147,4,2,21,0,3,...,ORIENTE,17.743489,-6.229444,-77.872778,0.767279,0.396175,0.542665,22.021474,11.698524,1.330599
4,325509701 2,1,1,1,96,4,2,21,0,3,...,ORIENTE,17.743489,-6.229444,-77.872778,0.767279,0.396175,0.542665,22.021474,11.698524,1.330599


## Se prepara los datos para el entrenamiento

In [ ]:
# df_master es tu DataFrame original
X_train, X_test, y_train, y_test = preparar_datos_bajo_peso(df_master)

# Verifica tamaños y proporciones
print(X_train.shape, X_test.shape)
print(y_train.value_counts(normalize=True), y_test.value_counts(normalize=True))
print(X_train["anio"].value_counts(normalize=True))
print(X_test["anio"].value_counts(normalize=True))

## Se Entrena el modelo

In [7]:
drop_columns_model = [
    "anio",
    "peso_bebe_nacimiento_gr",
    "id_hogar",
    "id_miembro_hogar",
    "id_cuestionario_mujer",
    "id_nacimiento",
    "id_miembro_hogar",
    "ubigeo",
    "anio",
    "latitud",
    "longitud",
    "latitude",
    "longitude",
]

In [ ]:
# Ejecutar el entrenamiento robusto
best_model, best_thresh, cv_metrics = train_bajo_peso(
    X=X_train, y=y_train, drop_cols=drop_columns_model, random_state=42
)

#Evaluar el modelo final en el conjunto de Test (nunca visto)
test_prob = best_model.predict_proba(X_test.drop(columns=drop_columns_model))[:, 1]

# 2. Crea una copia de X_test para trabajar de forma segura
df_evaluation = X_test.copy()
df_evaluation["real"] = y_test
df_evaluation["probability"] = test_prob

test_columns = {'real','probability'} 
columns_to_df_final= [x for x in list(df_evaluation.columns)  if x not in test_columns]
df_evaluation= df_evaluation[['id_hogar', 'id_miembro_hogar', 'id_cuestionario_mujer', 'id_nacimiento',
                               'real','probability']].merge(df_master[columns_to_df_final])



print( f"El mejor umbral para clasificar el modelo es : {best_thresh}" )

print(f"\nModelo final retornado: {type(best_model.named_steps['clf']).__name__}")

In [10]:
cv_metrics

,modelo,Precision,Recall,Specificity,F1,AUC_ROC,AUC_PR,MCC
0,Balanced RF,14.59%,36.86%,85.34%,20.36%,68.68%,13.78%,0.148127
1,LightGBM,7.43%,55.71%,54.66%,12.86%,56.17%,6.92%,0.052130
2,Regresión Logística,8.71%,54.58%,63.4%,15.0%,61.53%,8.69%,0.088403
3,XGBoost,16.35%,34.04%,88.66%,21.86%,70.31%,15.03%,0.163084


## Gráficando los facotes más determinantes

In [22]:
final_clf = best_model.named_steps["clf"]
feature_names = X_train.drop(columns=drop_columns_model).columns
if hasattr(final_clf, "feature_importances_"):
    importances = final_clf.feature_importances_
elif hasattr(final_clf, "coef_"):
    importances = np.abs(final_clf.coef_[0])
df_importance = pd.DataFrame({"feature": feature_names, "importance": importances}).sort_values(
    "importance", ascending=False
)


sns.set_theme(style="whitegrid", palette="muted")

fig, ax = plt.subplots(figsize=(12, 8))

# --- Gráfico: Feature Importance ---
top_features = df_importance.head(20)
sns.barplot(
    x="importance",
    y="feature",
    data=top_features,
    ax=ax,  
    palette="viridis",
    orient="h",
)

# --- Títulos y Etiquetas ---
ax.set_title("Factores Clave del Modelo (Feature Importance)", fontsize=18, weight="bold")
ax.set_xlabel("Nivel de Importancia", fontsize=14)
ax.set_ylabel("Característica", fontsize=14)
# Para un gráfico de barras horizontal, la grilla en el eje X es más útil
ax.grid(axis="x")


plt.tight_layout()
plt.savefig(project_root+"\\data\\03_reporting\\feature_importance.jpg", dpi=300, bbox_inches="tight")
plt.close(fig)

plt.show()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_2652\1375274713.py:18: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


## Guardando modelo y restultados

In [ ]:
## guardando el mejor modelo
with open(project_root+'\\data\\02_model_output\\best_model.pkl', 'wb') as archivo_salida:
    pickle.dump(best_model, archivo_salida)


excel_path = project_root + '\\data\\03_reporting\\metricas_resultados.xlsx'


with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:

    cv_metrics.to_excel(writer, sheet_name='metricas_modelo', index=False)
    df_evaluation.to_excel(writer, sheet_name='datos_con_probabilidad', index=False)
    df_importance.to_excel(writer, sheet_name='importancia_variables', index=False)